['P26', 'P569', 'P570', 'P108', 'P54', 'P286']

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from SPARQLWrapper import SPARQLWrapper


endpoint = SPARQLWrapper("http://114.212.81.217:8890/sparql/")

In [7]:
def get_all_item_with_relations_list(r_list,r_type="common",result_path="",t_type="period"):
    relation_stat=[]
    for relation in r_list:
        this_r_stat={}
        this_r_stat["r"]=relation
        cnt=0
        st_cnt=0
        en_cnt=0
        point_cnt=0
        st_en_pair_cnt=0
        if r_type=="common":
            query = '''
                PREFIX wd: <http://www.wikidata.org/entity/>
                PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                SELECT DISTINCT ?e1 ?e2 ?st ?en ?time
                WHERE
                {{
                    ?e1 p:{} ?s.
                    ?s ps:{} ?e2.
                    ?s a <http://wikiba.se/ontology#Statement>.

                    {{
                        ?s pq:P580 ?st.
                        OPTIONAL {{?s pq:P582 ?en.}}
                        OPTIONAL {{?s pq:P585 ?time.}}
                    }}
                    UNION
                    {{
                        ?s pq:P582 ?en.
                        OPTIONAL {{?s pq:P580 ?st.}}
                        OPTIONAL {{?s pq:P585 ?time.}}
                    }}
                    UNION
                    {{
                        ?s pq:P585 ?time.
                        OPTIONAL {{?s pq:P580 ?st.}}
                        OPTIONAL{{?s pq:P582 ?en.}}
                    }}
                }}
                '''.format(relation,relation)
        elif r_type=="time property":
            query = '''
                PREFIX wd: <http://www.wikidata.org/entity/>
                PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                SELECT DISTINCT ?e1 ?e2 ?st ?en
                WHERE
                {{
                    ?e1 p:{} ?s.
                    ?s ps:{} ?e2.

                    OPTIONAL {{?s pq:P580 ?st.}}
                    OPTIONAL {{?s pq:P582 ?en.}}
                }}
                '''.format(relation,relation)
        # print(query)
        endpoint.setQuery(query)
        endpoint.setReturnFormat(JSON)
        response = endpoint.query().convert()
        results = response['results']['bindings']
        
        query_res=[]
        print("%s Query complete!" % relation)
        for each_result in results:
            temporal_fact={}

            temporal_fact["s"]=each_result["e1"]["value"][31:]
            temporal_fact["p"]=relation

            if ("time" in each_result) and (t_type=="point"):
                if each_result["time"]["type"]=="typed-literal":
                    if each_result["time"]["datatype"]!="http://www.w3.org/2001/XMLSchema#dateTime":
                        print("DatatypeAlert!type:%s",each_result["en"]["datatype"])
                    temporal_fact["t1"]=each_result["time"]["value"].split("T")[0]
                    temporal_fact["t2"]=each_result["time"]["value"].split("T")[0]
                else:
                    continue
            elif t_type=="period":
                if "st" in each_result:
                    if each_result["st"]["type"]=="typed-literal":
                        if each_result["st"]["datatype"]!="http://www.w3.org/2001/XMLSchema#dateTime":
                            print("DatatypeAlert!type:%s",each_result["en"]["datatype"])
                        temporal_fact["t1"]=each_result["st"]["value"].split("T")[0]
                    else:
                        continue
                else:
                    temporal_fact["t1"]="null"
                
                if "en" in each_result:
                    if each_result["en"]["type"]=="typed-literal":
                        if each_result["en"]["datatype"]!="http://www.w3.org/2001/XMLSchema#dateTime":
                            print("DatatypeAlert!type:%s",each_result["en"]["datatype"])
                        temporal_fact["t2"]=each_result["en"]["value"].split("T")[0]
                    else:
                        continue
                else:
                    temporal_fact["t2"]="null"

                if (temporal_fact["t1"]=="null") and (temporal_fact["t2"]=="null"):
                    if "time" in each_result:
                        if each_result["time"]["type"]=="typed-literal":
                            if each_result["time"]["datatype"]!="http://www.w3.org/2001/XMLSchema#dateTime":
                                print("DatatypeAlert!type:%s",each_result["en"]["datatype"])
                            temporal_fact["t1"]=each_result["time"]["value"].split("T")[0]
                            temporal_fact["t2"]=each_result["time"]["value"].split("T")[0]
                        else:
                            continue
                    else:
                        continue
            if each_result["e2"]["type"]=="uri":
                temporal_fact["o"]=each_result["e2"]["value"][31:]
            elif each_result["e2"]["type"]=="typed-literal":
                if each_result["e2"]["datatype"]!="http://www.w3.org/2001/XMLSchema#dateTime":
                    print("DatatypeAlert!type:%s",each_result["en"]["datatype"])
                temporal_fact["o"]=each_result["e2"]["value"].split("T")[0]
                # if temporal_fact["t1"]=="null" and temporal_fact["t2"]=="null":
                temporal_fact["t1"]=temporal_fact["o"]
                temporal_fact["t2"]=temporal_fact["o"]
            else:
                continue

            if "st" in each_result:
                st_cnt+=1
            if "en" in each_result:
                en_cnt+=1
            if ("st" in each_result)and("en" in each_result):
                st_en_pair_cnt+=1
            if "time" in each_result:
                point_cnt+=1
            cnt+=1
            query_res.append(temporal_fact)

        this_r_stat["cnt"]=cnt
        this_r_stat["st_cnt"]=st_cnt
        this_r_stat["en_cnt"]=en_cnt
        this_r_stat["st_en_pair_ent"]=st_en_pair_cnt
        this_r_stat["point_cnt"]=point_cnt
        relation_stat.append(this_r_stat)
        f=open("%s/%sres.csv" %(result_path,relation),"w")
        for tf in query_res:
            f.write("%s,%s,%s,%s,%s\n" %(tf["s"],tf["p"],tf["o"],tf["t1"],tf["t2"]))
        f.close()
    return relation_stat

In [3]:
tr_file=open("property final.tsv")
line=tr_file.readline()
line=tr_file.readline()
optional_relation_list=[]
while line:
    relation=line.split('\t')[0][32:-1]
    optional_relation_list.append(relation)
    line=tr_file.readline()
print(optional_relation_list)

['P6', 'P17', 'P19', 'P20', 'P26', 'P27', 'P30', 'P35', 'P36', 'P37', 'P38', 'P39', 'P47', 'P50', 'P53', 'P54', 'P57', 'P58', 'P69', 'P85', 'P97', 'P102', 'P108', 'P119', 'P122', 'P127', 'P131', 'P150', 'P159', 'P161', 'P166', 'P170', 'P175', 'P180', 'P190', 'P197', 'P241', 'P276', 'P286', 'P411', 'P457', 'P463', 'P512', 'P521', 'P530', 'P551', 'P610', 'P612', 'P647', 'P669', 'P708', 'P710', 'P725', 'P726', 'P750', 'P802', 'P803', 'P859', 'P1038', 'P1066', 'P1075', 'P1344', 'P1366', 'P1399', 'P1411', 'P1433', 'P1435', 'P1448', 'P1596', 'P1640', 'P1652', 'P1789', 'P1891', 'P2238', 'P2291', 'P2567', 'P2568', 'P2632', 'P2652', 'P2828', 'P2838', 'P2962', 'P3085', 'P3148', 'P3300', 'P3342', 'P3448', 'P3919', 'P4002', 'P4675', 'P5030', 'P5460', 'P5817', 'P6087', 'P6364', 'P6872', 'P7047', 'P7152', 'P7779', 'P7936', 'P8047', 'P8138', 'P8413', 'P8764', 'P8791', 'P8839', 'P8852', 'P8938']


In [8]:
common_relation_list=['P26', 'P108', 'P54', 'P286']
timeobj_relation_list=['P569', 'P570']

stat_info=get_all_item_with_relations_list(common_relation_list,"common","raw/required/common/")
print("Phase 1 done.")

statfilename="PHASE1_RELATION_STAT.txt"
f=open(statfilename,"w")
f.write("?relation\t?count\t?st_count\t?en_count\t?st_en_pairs_count\t?point_count\n")
for this_r_stat in stat_info:
    f.write("%s\t%d\t%d\t%d\t%d\t%d\n" %(this_r_stat["r"],this_r_stat["cnt"],this_r_stat["st_cnt"],this_r_stat["en_cnt"],this_r_stat["st_en_pair_ent"],this_r_stat["point_cnt"]))
f.close()

P26 Query complete!
P108 Query complete!
P54 Query complete!
P286 Query complete!
Phase 1 done.


In [5]:
get_all_item_with_relations_list(timeobj_relation_list,"time property","raw/required/time_prop")
print("Phase 2 done.")

P569 Query complete!
P570 Query complete!
Phase 2 done.


In [10]:
stat_info=get_all_item_with_relations_list(optional_relation_list,"common","raw/optional/common")
print("Phase 3 done.")

statfilename="PHASE3_RELATION_STAT.txt"
f=open(statfilename,"w")
f.write("?relation\t?count\t?st_count\t?en_count\t?st_en_pairs_count\t?point_count\n")
for this_r_stat in stat_info:
    f.write("%s\t%d\t%d\t%d\t%d\t%d\n" %(this_r_stat["r"],this_r_stat["cnt"],this_r_stat["st_cnt"],this_r_stat["en_cnt"],this_r_stat["st_en_pair_ent"],this_r_stat["point_cnt"]))
f.close()

P6 Query complete!
P17 Query complete!
P19 Query complete!
P20 Query complete!
P26 Query complete!
P27 Query complete!
P30 Query complete!
P35 Query complete!
P36 Query complete!
P37 Query complete!
P38 Query complete!
P39 Query complete!
P47 Query complete!
P50 Query complete!
P53 Query complete!
P54 Query complete!
P57 Query complete!
P58 Query complete!
P69 Query complete!
P85 Query complete!
P97 Query complete!
P102 Query complete!
P108 Query complete!
P119 Query complete!
P122 Query complete!
P127 Query complete!
P131 Query complete!
P150 Query complete!
P159 Query complete!
P161 Query complete!
P166 Query complete!
P170 Query complete!
P175 Query complete!
P180 Query complete!
P190 Query complete!
P197 Query complete!
P241 Query complete!
P276 Query complete!
P286 Query complete!
P411 Query complete!
P457 Query complete!
P463 Query complete!
P512 Query complete!
P521 Query complete!
P530 Query complete!
P551 Query complete!
P610 Query complete!
P612 Query complete!
P647 Query com

In [14]:
#test common relation query sparql
relation="P1789"
query = '''
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT DISTINCT ?e1 ?e2 ?st ?en
    WHERE
    {{
        ?e1 p:{} ?s.
        ?s ps:{} ?e2.

        {{?s pq:P580 ?st.
        OPTIONAL {{?s pq:P582 ?en.}}}}
        UNION
        {{?s pq:P582 ?en.
        OPTIONAL {{?s pq:P580 ?st.}}}}
    }}
    '''.format(relation,relation)
# print(query)
endpoint.setQuery(query)
endpoint.setReturnFormat(JSON)
response = endpoint.query().convert()
results = response['results']['bindings']
print(results)
query_res=[]
print("%s Query complete!" % relation)
for each_result in results:
    temporal_fact={}
    temporal_fact["s"]=each_result["e1"]["value"][31:]
    temporal_fact["p"]=relation
    temporal_fact["o"]=each_result["e2"]["value"][31:]
    
    if "st" in each_result:
        if each_result["st"]["type"]=="typed-literal":
            if each_result["st"]["datatype"]!="http://www.w3.org/2001/XMLSchema#dateTime":
                print("DatatypeAlert!type:%s",each_result["en"]["datatype"])
            temporal_fact["t1"]=each_result["st"]["value"].split("T")[0]
        else:
            continue
    else:
        temporal_fact["t1"]="null"
    if "en" in each_result:
        if each_result["en"]["type"]=="typed-literal":
            if each_result["en"]["datatype"]!="http://www.w3.org/2001/XMLSchema#dateTime":
                print("DatatypeAlert!type:%s",each_result["en"]["datatype"])
            temporal_fact["t2"]=each_result["en"]["value"].split("T")[0]
        else:
            continue
    else:
        temporal_fact["t2"]="null"
    query_res.append(temporal_fact)

# f=open("%sres.csv" %relation,"w")
# for tf in query_res:
#     f.write("%s,%s,%s,%s,%s\n" %(tf["s"],tf["p"],tf["o"],tf["t1"],tf["t2"]))
# f.close()
# print(query_res)


[{'e1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q51709'}, 'e2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28912967'}, 'st': {'type': 'bnode', 'value': 'nodeID://b76002'}, 'en': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'value': '2016-01-13T00:00:00Z'}}, {'e1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q193701'}, 'e2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1557944'}, 'st': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'value': '2008-01-01T00:00:00Z'}}, {'e1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q156062'}, 'e2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2153'}, 'st': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'value': '1942-01-01T00:00:00Z'}}, {'e1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q56826397'}, 'e2': {'type': 'uri', 'value': 'http://www.wikidat

In [ ]:
#test timeobj relation query sparql
relation="P569"
query = '''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        SELECT  ?e1 \"{}\" ?time
        WHERE
        {{
            ?e1 wdt:{} ?time.
        }}
    '''.format(relation,relation)
# print(query)
endpoint.setQuery(query)
endpoint.setReturnFormat(JSON)
response = endpoint.query().convert()
results = response['results']['bindings']
print(results)
query_res=[]
print("%s Query complete!" % relation)
for each_result in results:
    if "literal" in each_result["time"]["type"]:
        temporal_fact={}
        temporal_fact["s"]=each_result["e1"]["value"][31:]
        temporal_fact["p"]=each_result["callret-1"]["value"]
        temporal_fact["o"]=each_result["time"]["value"]
        temporal_fact["t1"]=each_result["time"]["value"]
        temporal_fact["t2"]=each_result["time"]["value"]
        query_res.append(temporal_fact)
# f=open("%sres.csv" %relation,"w")
# for tf in query_res:
#     f.write("%s,%s,%s,%s,%s\n" %(tf["s"],tf["p"],tf["o"],tf["t1"],tf["t2"]))
# f.close()
# print(query_res)